# GT4Py's implementation of the three-dimensional dry isentropic model

## 1. Isentropic coordinates

Let $(x, y, \theta, t)$ be an isentropic, terrain-following coordinate system, where $x$ and $y$ are horizontal Cartesian coordinates, $\theta$ is the potential temperature, and $t$ is the time. The potential temperature is defined by
\begin{equation} 
    \theta = T \left( \dfrac{p}{p_{\text{ref}}} \right)^{R \, / \, c_p} \, ,
\end{equation}
with $T$ being the temperature, $p$ the pressure, $p_{\text{ref}} = 1000$ hPa, $R$ the gas constant, and $c_p$ the specific heat at constant pressure for dry air. In the isentropic reference system, the vertical velocity is given by
\begin{equation*} 
    \dot{\theta} = \dfrac{D \theta}{D t} \, ,
\end{equation*}
where $D / D t$ denotes the total (i.e., Lagrangian) derivative.

## 2. Governing equations in isentropic coordinates

Let us introduce the horizontal velocity vector $\boldsymbol{v} = (u, v)$, with $u$ and $v$ representing the $x$- and $y$-components, respectively, and the *isentropic density* $\sigma$,
\begin{equation*}
    \sigma = - \dfrac{1}{g} \dfrac{\partial p}{\partial \theta} \, ,
\end{equation*}
where $g$ is the gravitational constant. Upon assuming the atmosphere to be *adiabatic*, i.e.,
\begin{equation*}
    \dot{\theta} = 0 \, ,
\end{equation*}
the Euler equations for the isentropic density and the velocity components can be written in isentropic coordinates as:
\begin{align}
    \label{eq:continuity_nonconservative}
    \dfrac{\partial \sigma}{\partial t} + u \dfrac{\partial \sigma}{\partial x} + v \dfrac{\partial \sigma}{\partial y} & = 0 \, , \\
    \label{eq:x_momentum_nonconservative}
    \dfrac{\partial u}{\partial t} + u \dfrac{\partial u}{\partial x} + v \dfrac{\partial u}{\partial y} & = - \dfrac{\partial M}{\partial x} \, , \\
    \label{eq:y_momentum_nonconservative}
    \dfrac{\partial v}{\partial t} + u \dfrac{\partial v}{\partial x} + v \dfrac{\partial v}{\partial y} & = - \dfrac{\partial M}{\partial y} \, .
\end{align}
Here, $M$ is the Montgomery potential, defined by
\begin{equation*}
    M = c_p \, T + g \, z \, ,
\end{equation*}
with $z = z(x, y, \theta, t)$ the geometric height over sea level. The Montgomery potential occurs in the hydrostatic relation in isentropic coordinates, which read
\begin{equation}
    \label{eq:hydrostatic_relation}
    \dfrac{\partial M}{\partial \theta} = \Pi \, , \\
\end{equation}
where $\Pi$ is the Exner function, defined by
\begin{equation}
    \label{eq:exner_function}
    \Pi = c_p \left( \dfrac{p}{p_{\text{ref}}} \right)^{R / c_p} = \dfrac{c_p T}{\theta} \, .
\end{equation}
The hyperbolic system \eqref{eq:continuity_nonconservative}-\eqref{eq:y_momentum_nonconservative} is said to be in *non-conservative* form. The corresponding *conservative* version reads:
\begin{align}
    \label{eq:continuity_conservative}
    \dfrac{\partial \sigma}{\partial t} + \dfrac{\partial U}{\partial x} + \dfrac{\partial V}{\partial y} & = 0 \, , \\
    \label{eq:x_momentum_conservative}
    \dfrac{\partial U}{\partial t} + \dfrac{\partial u U}{\partial x} + \dfrac{\partial v U}{\partial y} & = - \sigma \dfrac{\partial M}{\partial x} \, , \\
    \label{eq:y_momentum_conservative}
    \dfrac{\partial V}{\partial t} + \dfrac{\partial u V}{\partial x} + \dfrac{\partial v V}{\partial y} & = - \sigma \dfrac{\partial M}{\partial y} \, .
\end{align}
where we have defined $U := \sigma u$ and $V := \sigma v$.

## 3. Numerical model

Consider a rectangular and uniform grid $(x_i, y_j, \theta_k)$ embedded in the $(x, y, \theta)$ computational domain $\Omega = [x_w, \, x_e] \times [y_s, \, y_n] \times [\theta_s, \, \theta_t]$. The location of the grid points is given by
\begin{equation*}
    \begin{aligned}
        x_i & = x_w + (i - 1) \Delta x = x_w + (i - 1) \dfrac{x_e - x_w}{N_x - 1} \, , && i = 1, \, \ldots \, , N_x \, , \\
        y_j & = y_s + (j - 1) \Delta y = y_s + (j - 1) \dfrac{y_n - y_s}{N_y - 1} \, , && j = 1, \, \ldots \, , N_y \, , \\
        \theta_k & = \theta_s + (k - 1/2) \Delta \theta = \theta_s + (k - 1/2) \dfrac{\theta_t - \theta_s}{N_{\theta}} \, , && k = 1, \, \ldots \, , N_{\theta} \, , \\
    \end{aligned}
\end{equation*}
Here, $N_x$, $N_y$ and $N_{\theta}$ are the number of grid points in the $x$-, $y$- and $\theta$-direction, respectively. The model variables are staggered on a $C$-grid, whereby:

-  the $x$-velocity $u$ is defined at the $x$-staggered grid points $(x_{i+1/2}, y_j, \theta_k)$, with $x_{i+1/2} = x_w + (i - 1/2) \Delta x$, $i = 0, \, \ldots \, , N_x$;
-  the $y$-velocity $v$ is defined at the $y$-staggered grid points $(x_i, y_{j+1/2}, \theta_k)$, with $y_{j+1/2} = y_s + (j - 1/2) \Delta y$, $j = 0, \, \ldots \, , N_y$;
-  the pressure and the Exner function are defined at the vertical half levels $\theta_{k+1/2} = \theta_s + k \Delta \theta$, $k = 0, \, \ldots \, , N_{\theta}$;
-  any other field is defined at the mass points $(x_i, y_j, \theta_k)$.

At the generic $n$-th timestep $t_n = n \Delta_t$, where $\Delta_t$ is the timestep size, the prognostic equations \eqref{eq:continuity_conservative}-\eqref{eq:y_momentum_conservative} are discretized using space- and time-centered finite differences:
\begin{align}
    \dfrac{s_{i,j,k}^{n+1} - s_{i,j,k}^{n-1}}{\Delta}
\end{align}

## 4. Simulation using GT4Py

In [ ]:
# Import built-in modules and classes
from datetime import datetime, timedelta
import os
import pickle
import sys
import time

# Import modules and classes from GT4Py and Tasmania libraries
import gridtools as gt
from tasmania.grids.grid_xyz import GridXYZ as Grid
from tasmania.dycore.dycore import DynamicalCore
from tasmania.model import Model


In [ ]:
# Create the grid
start = time.time()
grid = Grid(domain_x = [0, 500.e3], nx = 51, domain_y = [-250.e3, 250.e3], ny = 51, domain_z = [400, 300], nz = 50,
            units_x = 'm', dims_x = 'x', units_y = 'm', dims_y = 'm', units_z = 'K', dims_z = 'air_potential_temperature',
            topo_type = 'gaussian', topo_time = timedelta(seconds = 1800), topo_max_height = 1000., 
            topo_width_x = 50.e3, topo_width_y = 50.e3)
stop = time.time()
print('Grid created in {} ms.\n'.format((stop - start) * 1000.))

In [ ]:
# Instantiate the dycore
start = time.time()
dycore = DynamicalCore.factory(model = 'isentropic', time_scheme = 'centered', flux_scheme = 'centered', 
                               horizontal_boundary_type = 'relaxed', grid = grid, moist_on = False, backend = gt.mode.NUMPY, 
                               damp_on = True, damp_type = 'rayleigh', damp_depth = 15, damp_max = 0.0002,
                               smooth_on = True, smooth_type = 'first_order', smooth_coeff = 0.03)
stop = time.time()
print('Dycore instantiated in {} ms.\n'.format((stop-start) * 1000.))

In [ ]:
# Instantiate the model
start = time.time()
model = Model(dycore)
stop = time.time()
print('Model instantiated in {} ms.\n'.format((stop-start) * 1000.))

In [ ]:
# Compute the initial state
start = time.time()
state = dycore.get_initial_state(initial_time = datetime(year = 2018, month = 3, day = 15), initial_state_type = 0, 
                                 x_velocity_initial = 15., y_velocity_initial = 0., brunt_vaisala_initial = 0.01)
stop = time.time()
print('Initial state computed in {} ms.\n'.format((stop-start) * 1000.))

In [ ]:
# Let's run!
print('Start the simulation ...\n')
start = time.time()
state_out, state_save = model(dt = timedelta(seconds = 24), simulation_time = timedelta(hours = 12), state = state)
stop = time.time()
print('\nSimulation completed in {} s.\n'.format(stop-start))

## 5. Post-processing

In [ ]:
# Some post-processing: generate the contourf plot for the x-velocity at y = 0
state_save.contourf_xz(field_to_plot = 'x_velocity', y_level = 25, time_level = -1,
                       fontsize = 16, figsize = [7,8], title = '', 
                       x_factor = 1.e-3, x_label = '$x$ [km]',
                       z_factor = 1.e-3, z_label = '$z$ [km]', z_lim = [0,20], 
                       cmap_name = 'BuRd',
                       cbar_levels = 14, cbar_ticks_step = 2, cbar_center = 15., cbar_half_width = 6.5,
                       cbar_x_label = '$x$-velocity [m s$^{-1}$]', cbar_orientation = 'horizontal',
                       text = '$y = 0$', text_loc = 'upper right')

In [ ]:
# Some further post-processing: generate the contourf plot for the horizontal velocity at the surface layer
state_save.contourf_xy(field_to_plot = 'horizontal_velocity', z_level = -1, time_level = -1,
                       fontsize = 16, figsize = [7,8], title = '', 
                       x_factor = 1.e-3, x_label = '$x$ [km]',
                       y_factor = 1.e-3, y_label = '$y$ [km]', 
                       cmap_name = 'BuRd',
                       cbar_levels = 14, cbar_ticks_step = 2, cbar_center = 15., cbar_half_width = 6.5,
                       cbar_x_label = 'Horizontal velocity [m s$^{-1}$]', cbar_orientation = 'horizontal',
                       text = '$\\theta = \\theta_s$', text_loc = 'upper right')